In [6]:
import pandas as pd

In [7]:
scored_train_data = pd.read_csv('preped_train_data_wth_dummies.csv', index_col=0)


In [27]:
scored_train_data.groupby('User Key').count()

,Sale ID,Start At,log_Followers,log_Brand Appearance,log_Avg Price,Avg Discount,log_First Day Revenue,Conversion,Recency,Frequency,...,Expériences,Hygiène,Lingerie,Maroquinerie,Outdoor,Prêt-à-porter,Salon,Soins,Sportswear,interaction
User Key,,,,,,,,,,,,,,,,,,,,,
00036SsVCBaEYTtyO0CppxHLbZw2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
008hOZ6DmLQGyMf7W2yOP1skFSw1,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
00EgukEbzlPiX7BaAJHgXp92CqG3,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
00YdcE3QEDcBGLW9HFmwaPXOjNw1,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
00gWmYC2S3YFEQ9jYRTUyXUx1132,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzPTgtIXzFTigp7gNRDXQ49Cl7i2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
zzSlusermJRt7cMDuVrmT99YndD2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
zzWUNSYQ2CWwopiILtUU4kaBEm13,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5


In [31]:
scored_train_data[scored_train_data['User Key'] == 'zzWUNSYQ2CWwopiILtUU4kaBEm13']['Start At']

18295     2023-03-26
51342     2023-04-18
51343     2023-04-18
164272    2023-04-08
164273    2023-04-08
Name: Start At, dtype: object

In [18]:
user_information = scored_train_data[['User Key', 'Recency', 'Frequency', 'Monetary']]
available_sales = 

In [3]:
user_id = 'zzWUNSYQ2CWwopiILtUU4kaBEm13'

In [1]:
def scoring(user_id, user_information, available_sales, model):
    """ 
    Aims at returning ranked sales for a given user
    """

    user_features = user_information.columns
    sale_features = available_sales.columns

    user_specific_information = user_information[user_information['User Key'] == user_id]
    df_for_predictions = available_sales
    df_for_predictions[user_features] = user_specific_information

    return df_for_predictions